<a href="https://colab.research.google.com/github/jasleenkaursandhu/Reproducing-chest-xray-report-generation-boag/blob/main/random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Random Baseline Model for Report Generation
# This notebook implements a simple random baseline model for chest X-ray report generation.
# The baseline randomly selects a report from the training set for each test image

In [11]:
# Import necessary libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
import tqdm
from collections import defaultdict
import pickle
import gzip
import random
import re
import warnings
!pip install pydicom
import pydicom
from collections import Counter, defaultdict
from time import gmtime, strftime

In [12]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
base_path = '/content/drive/MyDrive/mimic-cxr-project'
!mkdir -p {base_path}/data
!mkdir -p {base_path}/output
!mkdir -p {base_path}/features

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Import the report parser module
import sys
sys.path.append(f"{base_path}/modules")
from report_parser import parse_report, MIMIC_RE
print("Successfully imported report parser module")

Successfully imported report parser module


In [14]:
# Load train and test data
data_dir = os.path.join(base_path, 'data')
files_path = os.path.join(base_path, 'files')
output_dir = os.path.join(base_path, 'output')
features_dir = os.path.join(base_path, 'features')

train_df = pd.read_csv(os.path.join(data_dir, 'train.tsv'), sep='\t')
test_df = pd.read_csv(os.path.join(data_dir, 'test.tsv'), sep='\t')

print(f"Train data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")

Train data shape: (824, 3)
Test data shape: (382, 3)


In [15]:
print("Implementing Random Baseline Model")

# Define the path to the files directory
files_path = os.path.join(base_path, 'files')

# Map each dicom to its corresponding report identifier
report_id_column = 'study_id'
if report_id_column in train_df.columns:
    report_lookup = dict(train_df[['dicom_id', report_id_column]].values)
    print(f"Created lookup using {report_id_column}")
else:
    print(f"Warning: {report_id_column} not found in columns: {train_df.columns.tolist()}")
    report_lookup = {}

print("Sample of lookup dictionary:")
print(dict(list(report_lookup.items())[:5]))

Implementing Random Baseline Model
Created lookup using study_id
Sample of lookup dictionary:
{'dfd9a06c-2994892e-f4a6bc1c-f6ec4803-283e5005': 55606773, 'ab50de59-ebf1d4df-20709b76-e8df1c5b-02561a38': 55244750, 'fafaee95-e11d24c5-ad39cfcd-302ac853-4a6f16ac': 58401243, 'fd53f5b0-0070f205-6c47cbe6-eaf7140d-12b09066': 57100718, '78e28f8c-fc928714-2cdc13f2-e6e45d40-89cb7eca': 53346921}


In [16]:
# Generate random reports for each test image
generated_reports = {}

for pred_dicom in tqdm.tqdm(test_df.dicom_id.values):
    found = False
    attempts = 0
    max_attempts = 100  # Limit attempts to avoid infinite loops

    while not found and attempts < max_attempts:
        attempts += 1

        # Randomly select a training image
        nearest_dicom = random.choice(train_df.dicom_id.values)

        if nearest_dicom not in report_lookup:
            continue

        report_id = report_lookup[nearest_dicom]

        # Get corresponding subject_id
        subject_row = train_df[train_df.dicom_id == nearest_dicom]
        if len(subject_row) == 0:
            continue

        subject_id = subject_row.iloc[0]['subject_id']

        # Construct path to the report
        subject_prefix = f"p{str(subject_id)[:2]}"
        subject_dir = f"p{subject_id}"
        study_dir = f"s{report_id}"
        report_file = f"{study_dir}.txt"
        report_path = os.path.join(files_path, subject_prefix, subject_dir, report_file)

        # Parse the report to extract sections
        try:
            if os.path.exists(report_path):
                report = parse_report(report_path)

                # If the report has a findings section, use it
                if 'findings' in report:
                    found = True
                    generated_reports[pred_dicom] = report['findings']
        except Exception as e:
            # Skip this report and try another
            continue

    if not found:
        print(f"Warning: Could not find a valid report for {pred_dicom} after {max_attempts} attempts")

print(f"Generated random reports for {len(generated_reports)}/{len(test_df)} test images")

100%|██████████| 382/382 [06:08<00:00,  1.04it/s]

Generated random reports for 382/382 test images


In [17]:
# Save the generated reports to a TSV file
pred_dir = os.path.join(base_path, 'output')
os.makedirs(pred_dir, exist_ok=True)

pred_file = os.path.join(pred_dir, 'random.tsv')
print(f"Saving predictions to {pred_file}")

with open(pred_file, 'w') as f:
    print('dicom_id\tgenerated', file=f)
    for dicom_id, generated in sorted(generated_reports.items()):
        # Escape any tab characters in the generated text
        cleaned_text = generated.replace('\t', ' ')
        print(f'{dicom_id}\t{cleaned_text}', file=f)

print(f"Saved random baseline predictions to {pred_file}")
print(f"Current time: {strftime('%Y-%m-%d %H:%M:%S', gmtime())}")

Saving predictions to /content/drive/MyDrive/mimic-cxr-project/output/random.tsv
Saved random baseline predictions to /content/drive/MyDrive/mimic-cxr-project/output/random.tsv
Current time: 2025-04-13 20:17:42


In [18]:
# Display sample of generated reports
sample_count = min(3, len(generated_reports))
sample_dicoms = list(generated_reports.keys())[:sample_count]

for dicom_id in sample_dicoms:
    print(f"\nSample report for {dicom_id}:")
    report_text = generated_reports[dicom_id]

    # Print preview of the report (first 200 characters)
    if len(report_text) > 200:
        print(report_text[:200] + "...")
    else:
        print(report_text)


Sample report for fea90b5d-a059ecc8-b5e68f8d-b7f33ed9-1d32d429:
aside from minimal bibasilar atelectasis, the lungs are clear. the hilar and cardiomediastinal contours are normal. there is no pneumothorax. there is no pleural effusion. pulmonary vascularity is nor...

Sample report for 06ba097a-bf8b917c-851d0e95-886936a1-90964781:
heart size is normal. the mediastinal and hilar contours are normal. the pulmonary vasculature is normal. bibasilar atelectasis, although minimal, appears somewhat increased compared to the previous s...

Sample report for 648449fa-5e173b2a-87663d57-4a0fbfc0-138e42c7:
lung volumes are somewhat low in the patient is rotated to the left. bronchovascular markings remain prominent. there is no focal consolidation. the heart and mediastinal structures are stable, allowi...
